In [2]:
import os
if os.getenv("CUDA_VISIBLE_DEVICES") is None:
    gpu_num = 0 # Use "" to use the CPU
    os.environ["CUDA_VISIBLE_DEVICES"] = f"{gpu_num}"
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
# tahtan
# Import Sionna
import sys
sys.path.append('../')
# sys.path.append('../pyaerial/src')
import sionna
# import pyaerial
# try:
#     import sionna
# except ImportError as e:
#     # Install Sionna if package is not already installed
#     import os
#     os.system("pip install sionna")
#     import sionna

import tensorflow as tf
# Configure the notebook to use only a single GPU and allocate only as much memory as needed
# For more details, see https://www.tensorflow.org/guide/gpu
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        tf.config.experimental.set_memory_growth(gpus[0], True)
    except RuntimeError as e:
        print(e)
# Avoid warnings from TensorFlow
tf.get_logger().setLevel('ERROR')

sionna.config.seed = 42 # Set seed for reproducible results

# Load the required Sionna components
from sionna.nr import PUSCHConfig, PUSCHTransmitter, PUSCHReceiver, CarrierConfig, PUSCHDMRSConfig,\
                        TBConfig, PUSCHPilotPattern, TBEncoder, PUSCHPrecoder, LayerMapper, LayerDemapper, check_pusch_configs,\
                        TBDecoder, PUSCHLSChannelEstimator
from sionna.nr.utils import *
from sionna.channel import AWGN, RayleighBlockFading, OFDMChannel, TimeChannel, time_lag_discrete_time_channel
from sionna.channel.utils import * 
from sionna.channel.tr38901 import Antenna, AntennaArray, UMi, UMa, RMa, TDL, CDL
from sionna.channel import gen_single_sector_topology as gen_topology
from sionna.utils import compute_ber, ebnodb2no, sim_ber, array_to_hash, create_timestamped_folders, b2b, f2f, BinarySource
from sionna.ofdm import KBestDetector, LinearDetector, MaximumLikelihoodDetector,\
        LSChannelEstimator, LMMSEEqualizer, RemoveNulledSubcarriers, ResourceGridDemapper,\
        ResourceGrid, ResourceGridMapper, OFDMModulator
from sionna.mimo import StreamManagement
from sionna.mapping import Mapper, Demapper


# from aerial.phy5g.ldpc import get_mcs
# from aerial.phy5g.ldpc import get_tb_size

In [3]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import time
from datetime import datetime, timedelta
# from bs4 import BeautifulSoup
import pickle
from collections import namedtuple
import json
from tqdm.notebook import tqdm
import itertools
import io
from typing import List, Tuple

## A Hello World Example

Let us start with a simple "Hello, World!" example in which we will simulate PUSCH transmissions from a single transmitter to a single receiver over an AWGN channel.

In [43]:
class MyPUSCHConfig(PUSCHConfig):
    def __init__(self):
        super().__init__(
            carrier_config=CarrierConfig(
                n_cell_id=0,
                cyclic_prefix="normal",
                subcarrier_spacing=30,
                n_size_grid=273,
                n_start_grid=0,
                slot_number=4,
                frame_number=0
            ),
            pusch_dmrs_config=PUSCHDMRSConfig(
                config_type=1,
                length=1,
                additional_position=1,
                dmrs_port_set=[0],
                n_id=0,
                n_scid=0,
                num_cdm_groups_without_data=2,
                type_a_position=2
            ),
            tb_config=TBConfig(
                channel_type='PUSCH',
                n_id=0,
                mcs_table=1,
                mcs_index=9
            ),
            mapping_type='A',
            n_size_bwp=273,
            n_start_bwp=0,
            num_layers=1,
            num_antenna_ports=1,
            precoding='non-codebook',
            tpmi=0,
            transform_precoding=False,
            n_rnti=2008,
            symbol_allocation=[0,14]
        )

_pusch_config_0 = MyPUSCHConfig()
_pusch_config_0.show()

_tb_size = _pusch_config_0.tb_size
_num_coded_bits = _pusch_config_0.num_coded_bits
_target_coderate = _pusch_config_0.tb.target_coderate
_num_bits_per_symbol = _pusch_config_0.tb.num_bits_per_symbol
_num_layers = _pusch_config_0.num_layers
_n_rnti = _pusch_config_0.n_rnti
_n_id = _pusch_config_0.tb.n_id

_dmrs_length = _pusch_config_0.dmrs.length
_dmrs_additional_position = _pusch_config_0.dmrs.additional_position
_num_cdm_groups_without_data = _pusch_config_0.dmrs.num_cdm_groups_without_data
_n_scid = _pusch_config_0.dmrs.n_scid
_n_id_n_scid = _pusch_config_0.dmrs.n_id[0]


Carrier Configuration
cyclic_prefix : normal
cyclic_prefix_length : 2.3437500000000002e-06
frame_duration : 0.01
frame_number : 0
kappa : 64.0
mu : 1
n_cell_id : 0
n_size_grid : 273
n_start_grid : 0
num_slots_per_frame : 20
num_slots_per_subframe : 2
num_symbols_per_slot : 14
slot_number : 4
sub_frame_duration : 0.001
subcarrier_spacing : 30
t_c : 5.086263020833334e-10
t_s : 3.2552083333333335e-08

PUSCH Configuration
dmrs_grid : shape (1, 3276, 14)
dmrs_grid_precoded : shape ()
dmrs_mask : shape (3276, 14)
dmrs_symbol_indices : [2, 11]
frequency_hopping : neither
l : [2, 11]
l_0 : 2
l_bar : [2, 11]
l_d : 14
l_prime : [0]
l_ref : 0
mapping_type : A
n : shape (819,)
n_rnti : 2008
n_size_bwp : 273
n_start_bwp : 0
num_antenna_ports : 1
num_coded_bits : 78624
num_layers : 1
num_ov : 0
num_res_per_prb : 144
num_resource_blocks : 273
num_subcarriers : 3276
precoding : non-codebook
precoding_matrix : None
symbol_allocation : [0, 14]
tb_size : 52224
tpmi : 0
transform_precoding : False

PUSCH 

In [50]:
_pusch_config = MyPUSCHConfig()
_pusch_config.tb.mcs_index = 0
_pusch_config.show()

Carrier Configuration
cyclic_prefix : normal
cyclic_prefix_length : 2.3437500000000002e-06
frame_duration : 0.01
frame_number : 0
kappa : 64.0
mu : 1
n_cell_id : 0
n_size_grid : 273
n_start_grid : 0
num_slots_per_frame : 20
num_slots_per_subframe : 2
num_symbols_per_slot : 14
slot_number : 4
sub_frame_duration : 0.001
subcarrier_spacing : 30
t_c : 5.086263020833334e-10
t_s : 3.2552083333333335e-08

PUSCH Configuration
dmrs_grid : shape (1, 3276, 14)
dmrs_grid_precoded : shape ()
dmrs_mask : shape (3276, 14)
dmrs_symbol_indices : [2, 11]
frequency_hopping : neither
l : [2, 11]
l_0 : 2
l_bar : [2, 11]
l_d : 14
l_prime : [0]
l_ref : 0
mapping_type : A
n : shape (819,)
n_rnti : 2008
n_size_bwp : 273
n_start_bwp : 0
num_antenna_ports : 1
num_coded_bits : 78624
num_layers : 1
num_ov : 0
num_res_per_prb : 144
num_resource_blocks : 273
num_subcarriers : 3276
precoding : non-codebook
precoding_matrix : None
symbol_allocation : [0, 14]
tb_size : 9216
tpmi : 0
transform_precoding : False

PUSCH D

In [ ]:
from dataclasses import dataclass
from dataclasses import field
# NUM_RE_PER_PRB = 12
# NUM_PRB_MAX = 273
# NUM_SYMBOLS = 14
# MAX_NUM_CODE_BLOCKS = 152
# MAX_DL_LAYERS = 16

@dataclass
class PuschUeConfig:
    """A class holding all dynamic PUSCH parameters for a single slot, single UE.

    Args:
        scid (int): DMRS sequence initialization [TS38.211, sec 7.4.1.1.2].
        layers (int): Number of layers.
        dmrs_ports (int): Allocated DMRS ports.
        rnti (int): The 16-bit RNTI value of the UE.
        data_scid (List[int]): Data scrambling ID, more precisely `dataScramblingIdentityPdsch`
            [TS38.211, sec 7.3.1.1].
        mcs_table (int): MCS table to use (see TS 38.214).
        mcs_index (int): MCS index to use.
        code_rate (int): Code rate, expressed as the number of information
            bits per 1024 coded bits expressed in 0.1 bit units.
        mod_order (int): Modulation order.
        tb_size (int): TB size in bytes.
        rv (List[int]): Redundancy version.
        ndi (List[int]): New data indicator.
    """
    scid: int = 0
    layers: int = 1
    dmrs_ports: int = 1
    rnti: int = 1
    data_scid: int = 41
    mcs_table: int = 0
    mcs_index: int = 0
    code_rate: int = 1930
    mod_order: int = 2
    tb_size: int = 96321
    rv: int = 0
    ndi: int = 1
    harq_process_id: int = 0


@dataclass
class PuschConfig:
    """A class holding all dynamic PUSCH parameters for a single slot, single UE group.

    Args:
        num_dmrs_cdm_grps_no_data (int): Number of DMRS CDM groups without data
            [3GPP TS 38.212, sec 7.3.1.1]. Value: 1->3.
        dmrs_scrm_id (int): DMRS scrambling ID.
        start_prb (int): Start PRB index of the UE group allocation.
        num_prbs (int): Number of allocated PRBs for the UE group.
        prg_size (int): The Size of PRG in PRB for PUSCH per-PRG channel estimation.
        num_ul_streams (int): The number of active streams for this PUSCH.
        dmrs_syms (List[int]): For the UE group, a list of binary numbers each indicating whether
            the corresponding symbol is a DMRS symbol.
        dmrs_max_len (int): The `maxLength` parameter, value 1 or 2, meaning that DMRS are
            single-symbol DMRS or single- or double-symbol DMRS. Note that this needs to be
            consistent with `dmrs_syms`.
        dmrs_add_ln_pos (int): Number of additional DMRS positions.  Note that this needs to be
            consistent with `dmrs_syms`.
        start_sym (int): Start OFDM symbol index for the UE group allocation.
        num_symbols (int): Number of symbols in the UE group allocation.
    """
    # UE parameters.
    ue_configs: List[PuschUeConfig]

    # UE group parameters.
    num_dmrs_cdm_grps_no_data: int = 2
    dmrs_scrm_id: int = 41
    start_prb: int = 0
    num_prbs: int = 273
    prg_size: int = 1
    num_ul_streams: int = 1
    dmrs_syms: List[int] = \
        field(default_factory=lambda: [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0])
    dmrs_max_len: int = 2
    dmrs_add_ln_pos: int = 1
    start_sym: int = 2
    num_symbols: int = 12


PuschConfig(ue_configs=[PuschUeConfig(scid=0, layers=1, dmrs_ports=1, rnti=1, data_scid=41, mcs_table=0, mcs_index=0, code_rate=1930, mod_order=2, tb_size=96321, rv=0, ndi=1, harq_process_id=0)], num_dmrs_cdm_grps_no_data=2, dmrs_scrm_id=41, start_prb=0, num_prbs=273, prg_size=1, num_ul_streams=1, dmrs_syms=[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0], dmrs_max_len=2, dmrs_add_ln_pos=1, start_sym=2, num_symbols=12)

In [53]:
# Simulation parameters.
esno_db_range = np.arange(-5.4, -4.4, 0.2)
num_slots = 10000
min_num_tb_errors = 250

# Numerology and frame structure. See TS 38.211.
num_ofdm_symbols = 14
fft_size = 4096
cyclic_prefix_length = 288
subcarrier_spacing = 30e3
num_guard_subcarriers = (410, 410)
num_slots_per_frame = 20

# System/gNB configuration
num_tx_ant = 1             # UE antennas
num_rx_ant = 8             # gNB antennas
cell_id = 41               # Physical cell ID
enable_pusch_tdi = 0       # Enable time interpolation for equalizer coefficients
eq_coeff_algo = 1          # Equalizer algorithm

# PUSCH configuration
rnti = 1234                # UE RNTI
scid = 0                   # DMRS scrambling ID
data_scid = 0              # Data scrambling ID
layers = 1                 # Number of layers
mcs_index = 0              # MCS index as per TS 38.214 table. Note: Es/No range may need to be changed too to get meaningful results.
mcs_table = 1              # MCS table index
dmrs_ports = 1             # Used DMRS port.
start_prb = 0              # Start PRB index.
num_prbs = 273             # Number of allocated PRBs.
start_sym = 0              # Start symbol index.
num_symbols = 14           # Number of symbols.
dmrs_scrm_id = 41          # DMRS scrambling ID
dmrs_syms = [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0]  # Indicates which symbols are used for DMRS.
dmrs_max_len = 1
dmrs_add_ln_pos = 1
num_dmrs_cdm_grps_no_data = 2
# mod_order, code_rate = get_mcs(mcs_index, mcs_table)  # Different indexing for MCS table. # The code rate here equals code_rate/1024
mod_order, code_rate = select_mcs(mcs_index=mcs_index,table_index=mcs_table) # The code rate here equals code_rate

num_dmrs_syms = sum(dmrs_syms)
# tb_size = get_tb_size(mod_order, code_rate, dmrs_syms, num_prbs, start_sym, num_symbols, layers)  # TB size in bits
tb_size, cb_size, num_cbs, cw_length, tb_crc_length, cb_crc_length \
        = calculate_tb_size(modulation_order=mod_order, 
                            target_coderate=code_rate,
                            num_prbs=num_prbs,
                            num_ofdm_symbols=num_symbols,
                            num_dmrs_per_prb=6*num_dmrs_cdm_grps_no_data*num_dmrs_syms,  # 6 for DMRS configuration type 1, and 4 for type 2.
                            num_layers=layers,
                            verbose=False) 

# Channel parameters
carrier_frequency = 3.5e9  # Carrier frequency in Hz.                    
delay_spread = 100e-9      # Nominal delay spread in [s]. Please see the CDL documentation
                           # about how to choose this value.
link_direction = "uplink"
channel_model = "Rayleigh" # Channel model: Suitable values:
                           # "Rayleigh" - Rayleigh block fading channel model (sionna.channel.RayleighBlockFading)
                           # "CDL-x", where x is one of ["A", "B", "C", "D", "E"] - for 3GPP CDL channel models
                           #          as per TR 38.901.
speed = 0.8333             # UE speed [m/s]. The direction of travel will chosen randomly within the x-y plane.

In [57]:
pusch_ue_config = PuschUeConfig(
    scid=scid,
    layers=layers,
    dmrs_ports=dmrs_ports,
    rnti=rnti,
    data_scid=data_scid,
    mcs_table=mcs_table,
    mcs_index=mcs_index,
    code_rate=int(code_rate * 10),
    mod_order=mod_order,
    tb_size=tb_size // 8
)

pusch_configs = [PuschConfig(
    ue_configs=[pusch_ue_config],
    num_dmrs_cdm_grps_no_data=num_dmrs_cdm_grps_no_data,
    dmrs_scrm_id=dmrs_scrm_id,
    start_prb=start_prb,
    num_prbs=num_prbs,
    dmrs_syms=dmrs_syms,
    dmrs_max_len=dmrs_max_len,
    dmrs_add_ln_pos=dmrs_add_ln_pos,
    start_sym=start_sym,
    num_symbols=num_symbols
)]

pusch_configs

[PuschConfig(ue_configs=[PuschUeConfig(scid=0, layers=1, dmrs_ports=1, rnti=1234, data_scid=0, mcs_table=1, mcs_index=0, code_rate=1, mod_order=2, tb_size=1152, rv=0, ndi=1, harq_process_id=0)], num_dmrs_cdm_grps_no_data=2, dmrs_scrm_id=41, start_prb=0, num_prbs=273, prg_size=1, num_ul_streams=1, dmrs_syms=[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0], dmrs_max_len=1, dmrs_add_ln_pos=1, start_sym=0, num_symbols=14)]

In [9]:
"""
frame_number, n_cell_id: No change
n_rnti, tb.n_id: Change in tb_encoder, but not in dmrs (same c -> same x)
slot_number, dmrs.n_id, dmrs.n_scid, mcs: No change in tb_encoder (same b -> same c), but in dmrs
"""

'\nframe_number, n_cell_id: No change\nn_rnti, tb.n_id: Change in tb_encoder, but not in dmrs (same c -> same x)\nslot_number, dmrs.n_id, dmrs.n_scid, mcs: No change in tb_encoder (same b -> same c), but in dmrs\n'